In [1]:
import pandas as pd
from pandas import read_csv
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import sklearn.cross_validation
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.width', 100)
pd.set_option('precision', 3)

In [3]:
train_file_name = r'data/hackerrank_email/training_dataset.csv'
data = pd.read_csv(train_file_name)
print(data.shape)
data.head(10)

(486048, 54)


,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.464e+09,NaN,NaN,1.460e+09,1432533023,...,0,0,0,0,13,0,0,13,0,False
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461e+09,1432184291,...,0,3,16,0,83,0,43,83,4,False
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463e+09,1433045937,...,0,0,0,0,16,0,3,16,0,False
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.463e+09,1432184291,...,0,0,16,0,85,0,9,85,0,False
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461e+09,1432998058,...,0,1,17,0,43,0,0,43,0,False
5,UREtRIcAzBFKmMt9ZSkFLofOWKAKmzsDD7BwrdxcPuk=,UGATDXARg7jMEInKH7oXgty2nwxnwD2l0OW/8Nsa0MI=,mail_category_6,mail_type_1,1459927021,NaN,NaN,NaN,1.458e+09,1431945704,...,0,0,13,0,88,0,0,88,0,False
6,NiDHS4AU/QOs3ppF618O6bUg8BxJd+EV4Ek26KWrDc0=,qE9zgWiITYA97RUiN4X/t9IVWLViLz+lKijaYegyBiQ=,mail_category_1,mail_type_1,1460230930,1.460e+09,NaN,NaN,1.460e+09,1432658834,...,0,0,0,0,20,0,12,20,12,False
7,sGNmP4gGVwh3YBpXtq6ZI05PIa3Z0iUaF+ujKonctu0=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1462282973,NaN,NaN,NaN,1.462e+09,1432294499,...,0,0,0,0,14,0,14,14,14,False
8,nyoFu8qU0B5viJDXfrVRQ65ytOSSSY3c7okcjo1Z76M=,4+EEK8RbNYwuFCHznY9XSRCV4Yek60bHVgnP3jtjjzk=,mail_category_13,mail_type_2,1462888545,1.463e+09,NaN,NaN,1.463e+09,1432795707,...,0,19,265,0,129,0,0,129,0,False
9,A2EUbD7QN5SXpJ7FhPY5Ne7zwjwYhRvAheGskLl0Os8=,4+EEK8RbNYwuFCHznY9XSRCV4Yek60bHVgnP3jtjjzk=,mail_category_13,mail_type_2,1462888549,NaN,NaN,NaN,1.462e+09,1432059719,...,0,0,0,0,3,0,0,3,0,False


In [4]:
test_file_name = r'data/hackerrank_email/test_dataset.csv'
test_data = pd.read_csv(test_file_name)
test_data.shape

(207424, 48)

In [5]:
list(set(data.columns) - set(test_data.columns))

['clicked',
 'unsubscribe_time',
 'click_time',
 'opened',
 'unsubscribed',
 'open_time']

In [6]:
for col in ['user_id','mail_id','clicked','unsubscribe_time','click_time','unsubscribed','open_time']:
    data.drop(col, axis=1, inplace=True)

In [7]:
data['mail_category'] = pd.Categorical(data.mail_category).codes
data['mail_type'] = pd.Categorical(data.mail_type).codes

In [8]:
data.opened = data.opened.astype('int')
data.hacker_confirmation = data.hacker_confirmation.astype('int')

In [9]:
data.fillna(0, inplace=True)

In [10]:
data.head(10)

,mail_category,mail_type,sent_time,last_online,hacker_created_at,hacker_timezone,contest_login_count,contest_login_count_1_days,contest_login_count_30_days,contest_login_count_365_days,...,submissions_count_contest,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days
0,10,0,1463497837,1.460e+09,1432533023,18000.0,1,0,0,1,...,0,0,0,0,0,13,0,0,13,0
1,0,0,1461357640,1.461e+09,1432184291,-25200.0,3,0,1,3,...,16,0,3,16,0,83,0,43,83,4
2,0,0,1463499639,1.463e+09,1433045937,18000.0,3,0,0,3,...,0,0,0,0,0,16,0,3,16,0
3,0,0,1463182983,1.463e+09,1432184291,-25200.0,3,0,0,3,...,16,0,0,16,0,85,0,9,85,0
4,12,0,1461855019,1.461e+09,1432998058,18000.0,5,0,0,5,...,17,0,1,17,0,43,0,0,43,0
5,14,0,1459927021,1.458e+09,1431945704,18000.0,2,0,0,2,...,13,0,0,13,0,88,0,0,88,0
6,0,0,1460230930,1.460e+09,1432658834,3600.0,1,0,0,1,...,0,0,0,0,0,20,0,12,20,12
7,0,0,1462282973,1.462e+09,1432294499,18000.0,1,0,0,1,...,0,0,0,0,0,14,0,14,14,14
8,4,1,1462888545,1.463e+09,1432795707,18000.0,53,1,5,53,...,265,0,19,265,0,129,0,0,129,0
9,4,1,1462888549,1.462e+09,1432059719,-25200.0,1,0,0,1,...,0,0,0,0,0,3,0,0,3,0


In [11]:
#cols = [col for col in data.columns if col not in ['opened']]
#print(cols)
array = data.values
#X = array[:,:]
Y = data['opened'].values
data.drop('opened', axis=1, inplace=True)
X = data.values
#temp = data[cols]


#temp = data['~opened']
#temp.drop('opened', axis=1, inplace=True)
#X = temp.values[:,:]
#remove opened before assigning x

In [12]:
data.columns

Index([u'mail_category', u'mail_type', u'sent_time', u'last_online', u'hacker_created_at',
       u'hacker_timezone', u'contest_login_count', u'contest_login_count_1_days',
       u'contest_login_count_30_days', u'contest_login_count_365_days',
       u'contest_login_count_7_days', u'contest_participation_count',
       u'contest_participation_count_1_days', u'contest_participation_count_30_days',
       u'contest_participation_count_365_days', u'contest_participation_count_7_days',
       u'forum_comments_count', u'forum_count', u'forum_expert_count', u'forum_questions_count',
       u'hacker_confirmation', u'ipn_count', u'ipn_count_1_days', u'ipn_count_30_days',
       u'ipn_count_365_days', u'ipn_count_7_days', u'ipn_read', u'ipn_read_1_days',
       u'ipn_read_30_days', u'ipn_read_365_days', u'ipn_read_7_days', u'submissions_count',
       u'submissions_count_1_days', u'submissions_count_30_days', u'submissions_count_365_days',
       u'submissions_count_7_days', u'submissions_coun

In [13]:
features = []
features.append(('pca', PCA(n_components=3)))
features.append(('select_best', SelectKBest(k=6)))
feature_union = FeatureUnion(features)

In [14]:
estimators = []
estimators.append(('feature_union', feature_union))
#estimators.append(('logistic', LogisticRegression()))
#estimators.append(('dtc', DecisionTreeClassifier()))
estimators.append(('rfc', RandomForestClassifier(n_estimators=100, max_features=6)))
#model = Pipeline(estimators)

In [15]:
model = RandomForestClassifier(n_estimators=20, max_features=6)

In [16]:
num_folds = 10
num_instances = len(X)
seed = 7
kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results)

[ 0.6866166   0.65201111  0.63923465  0.646559    0.65415081  0.64532456
  0.62328979  0.64106573  0.69813184  0.6785244 ]


In [17]:
print("Scoring - Accuracy: %.3f (%.3f)") % (results.mean(), results.std())

Scoring - Accuracy: 0.656 (0.022)


In [18]:
test_size = 0.33
train_test_seed = 7
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=test_size, random_state=train_test_seed)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)

In [19]:
pd.DataFrame(matrix, index=['0 (Actual)', '1 (Actual)'], columns=['0 (Predicted)', '1 (Predicted)'])

,0 (Predicted),1 (Predicted)
0 (Actual),94939,12465
1 (Actual),28399,24593


In [20]:
test_some_x = X[:500]
test_some_y = Y[:500]
test_some_predicted = model.predict(test_some_x)

In [21]:
print(test_some_predicted)

[1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 1 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 1 0 0 0 0 

In [22]:
print(test_some_y)

[1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0
 0 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0
 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0
 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 1 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0
 0 0 1 1 0 1 0 1 0 0 0 0 

In [23]:
print(len(X))

486048


In [24]:
for col in ['user_id','mail_id']:
    test_data.drop(col, axis=1, inplace=True)

In [25]:
test_data['mail_category'] = pd.Categorical(test_data.mail_category).codes
test_data['mail_type'] = pd.Categorical(test_data.mail_type).codes

In [26]:
test_data.hacker_confirmation = test_data.hacker_confirmation.astype('int')
test_data.fillna(0, inplace=True)

In [27]:
#list(set(data.columns) - set(test_data.columns))


In [28]:
test_predicted = model.predict(test_data)

In [29]:
print(test_predicted[:1000])

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 

In [30]:
#test_predicted.to_csv('open_email_test_output.csv', header=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [32]:
#import numpy as np
#np.savetxt('np_open_email_test_output.csv', test_predicted, delimiter=',')   # X is an array

In [41]:
test_predicted_df = pd.DataFrame(test_predicted)
#test_predicted_df.head(30)     
test_predicted_df.to_csv('df_open_email_test_output_65.csv', header=False, index=False)

In [43]:
test_predicted_df.shape

(207424, 1)

In [44]:
test_data.shape

(207424, 46)